## Xception model training

### import libs

In [1]:
import os
from PIL import Image
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.utils.training_utils import multi_gpu_model
from keras import optimizers
from keras import regularizers
from multiprocessing import cpu_count
import tensorflow as tf

Using TensorFlow backend.


In [2]:
class ZeroMeanCenter():
    def __init__(self, mean=None):
        if mean is not None:
            mean = np.asarray(Image.open(mean))
        self.mean = mean
        
    def __call__(self, img):
        if self.mean is not None:
            img -= self.mean
        return img

In [3]:
nb_classes = 21
nb_cpus = 16
nb_gpus = 8
os.environ["CUDA_VISIBLE_DEVICES"] = '0, 1, 2, 3, 4, 5, 6, 7'

image_size = (299, 299)
input_shape= (299, 299, 3)

train_path = "/home/ssd_array0/Data/batch6.4_cells/CELLS-half/train"
valid_path = "/home/ssd_array0/Data/batch6.4_cells/CELLS-half/valid"

### pretrain model

In [4]:
with tf.device('/cpu:0'):
    input_tensor = Input(input_shape)
    x = Lambda(xception.preprocess_input)(input_tensor)

    base_model = Xception(input_tensor=x, weights='imagenet', include_top=False)
    m_out = base_model.output
    p_out = GlobalAveragePooling2D()(m_out)
    p_out = Dropout(0.5)(p_out)
    predictions = Dense(nb_classes, activation='softmax', name="predictions")(p_out)

    for layer in base_model.layers:
        layer.trainable = False

    model = Model(inputs=base_model.input, outputs=predictions)

if nb_gpus > 1:
    parallel_model = multi_gpu_model(model, gpus=nb_gpus)
else:
    parallel_model = model
    
optimizer = optimizers.SGD(lr=0.005, momentum=0.9, decay=0.0003)
parallel_model.compile(optimizer=optimizer, 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
model.summary()

83689472/83683744 [==============================] - 14s 0us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 299, 299, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         lambda_1[0][0]                   
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
___________________________________________

In [5]:
batch_size = 256
epochs = 1

In [6]:
zmc = ZeroMeanCenter(mean='average.bmp')
img_gen_t = ImageDataGenerator(preprocessing_function=zmc)
train_generator = img_gen_t.flow_from_directory(train_path, 
                                                target_size=image_size, shuffle=True, batch_size=batch_size)

img_gen_v = ImageDataGenerator(preprocessing_function=zmc)
valid_generator = img_gen_v.flow_from_directory(valid_path,
                                                target_size=image_size, shuffle=True, batch_size=batch_size)

parallel_model.fit_generator(generator=train_generator, 
                             steps_per_epoch=len(train_generator), 
                             epochs=epochs, 
                             verbose=1,
                             validation_data=valid_generator, 
                             validation_steps=len(valid_generator), 
                             workers=nb_cpus, 
                             use_multiprocessing=True)

model.save_weights("Xception_first_train.h5")

Found 1787040 images belonging to 21 classes.
Found 49608 images belonging to 21 classes.
Epoch 1/1
6981/6981 [==============================] - 6500s 931ms/step - loss: 1.4242 - acc: 0.5717 - val_loss: 3.0761 - val_acc: 0.2191


### train model

In [7]:
with tf.device('/cpu:0'):
    input_tensor = Input(input_shape)
    x = Lambda(xception.preprocess_input)(input_tensor)

    base_model = Xception(input_tensor=x, weights=None, include_top=False)
    m_out = base_model.output
    p_out = GlobalAveragePooling2D()(m_out)
    p_out = Dropout(0.5)(p_out)
    predictions = Dense(nb_classes, activation='softmax', name="predictions")(p_out)

    model = Model(inputs=base_model.input, outputs=predictions)
    
    model.load_weights("Xception_first_train.h5")

if nb_gpus > 1:
    parallel_model = multi_gpu_model(model, gpus=nb_gpus)
else:
    parallel_model = model
    
optimizer = optimizers.SGD(lr=0.005, momentum=0.9, decay=0.0003)
parallel_model.compile(optimizer=optimizer, 
                       loss='categorical_crossentropy', 
                       metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
lambda_10 (Lambda)              (None, 299, 299, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         lambda_10[0][0]                  
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_con

In [ ]:
# parallel_model.load_weights('batch6.4_007_0.0655.hdf5')

In [8]:
batch_size = 128
epochs = 100

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard

zmc = ZeroMeanCenter(mean='average.bmp')
img_gen_t = ImageDataGenerator(preprocessing_function=zmc)
train_generator = img_gen_t.flow_from_directory(train_path, 
                                                target_size=image_size, shuffle=True, batch_size=batch_size)

img_gen_v = ImageDataGenerator(preprocessing_function=zmc)
valid_generator = img_gen_v.flow_from_directory(valid_path,
                                                target_size=image_size, shuffle=True, batch_size=batch_size)

checkpoint = ModelCheckpoint("batch6.4_{epoch:03d}_{val_loss:.4f}.hdf5", monitor='val_loss', verbose=1,
                             save_best_only=False, save_weights_only=True, mode='min', period=1)

tensorboard = TensorBoard(log_dir="./logs", histogram_freq=0, batch_size=batch_size, write_graph=True, write_images=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=0.000001)

callbacks = [checkpoint, tensorboard, reduce_lr]


parallel_model.fit_generator(generator=train_generator, 
                             steps_per_epoch=len(train_generator), 
                             epochs=epochs, 
                             verbose=1,
                             validation_data=valid_generator, 
                             validation_steps=len(valid_generator), 
                             callbacks=callbacks, 
                             workers=nb_cpus, 
                             use_multiprocessing=True, 
#                              initial_epoch=3
                             )

Found 1787040 images belonging to 21 classes.
Found 49608 images belonging to 21 classes.
Epoch 1/100
 1731/13962 [==>...........................] - ETA: 1:52:10 - loss: 0.4057 - acc: 0.8587

In [ ]:
# model.save_weights("batch6.4_train3_finetuned.h5")